In [1]:
import os
import numpy as np

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

from test_helper import Test
import os.path

fileName = os.path.join('D:/', 'DM_CURSO.CSV')

numPartitions = 100
CursoRawData = sc.textFile(fileName, numPartitions, 8)

In [25]:
# pega amostra e mostra tela
numPoints = CursoRawData.count()
print numPoints
samplePoints = CursoRawData.take(1)
print samplePoints

# começa limpeza dos dados
from pyspark.mllib.regression import LabeledPoint
import numpy as np


header = CursoRawData.take(1)[0] # o cabeçalho é a primeira linha do arquivo
print header

34696
[u'CO_IES|NO_IES|CO_CATEGORIA_ADMINISTRATIVA|DS_CATEGORIA_ADMINISTRATIVA|CO_ORGANIZACAO_ACADEMICA|DS_ORGANIZACAO_ACADEMICA|CO_LOCAL_OFERTA_CURSO|CO_MUNICIPIO_CURSO|NO_MUNICIPIO_CURSO|CO_UF_CURSO|SGL_UF_CURSO|NO_REGIAO_CURSO|IN_CAPITAL_CURSO|CO_CURSO|NO_CURSO|CO_SITUACAO_CURSO|DS_SITUACAO_CURSO|CO_OCDE|NO_OCDE|CO_OCDE_AREA_GERAL|NO_OCDE_AREA_GERAL|CO_OCDE_AREA_ESPECIFICA|NO_OCDE_AREA_ESPECIFICA|CO_OCDE_AREA_DETALHADA|NO_OCDE_AREA_DETALHADA|CO_GRAU_ACADEMICO|DS_GRAU_ACADEMICO|CO_MODALIDADE_ENSINO|DS_MODALIDADE_ENSINO|CO_NIVEL_ACADEMICO|DS_NIVEL_ACADEMICO|IN_GRATUITO|TP_ATRIBUTO_INGRESSO|NU_CARGA_HORARIA|DT_INICIO_FUNCIONAMENTO|DT_AUTORIZACAO_CURSO|IN_AJUDA_DEFICIENTE|IN_MATERIAL_DIGITAL|IN_MATERIAL_AMPLIADO|IN_MATERIAL_TATIL|IN_MATERIAL_IMPRESSO|IN_MATERIAL_AUDIO|IN_MATERIAL_BRAILLE|IN_MATERIAL_LIBRAS|IN_DISCIPLINA_LIBRAS|IN_TRADUTOR_LIBRAS|IN_GUIA_INTERPRETE|IN_RECURSOS_COMUNICACAO|IN_RECURSOS_INFORMATICA|IN_INTEGRAL_CURSO|IN_MATUTINO_CURSO|IN_VESPERTINO_CURSO|IN_NOTURNO_CURSO|NU_

In [47]:
import re
import datetime
from collections import namedtuple

headeritems = header.split('|') # transformar o cabeçalho em lista
print headeritems


[u'CO_IES', u'NO_IES', u'CO_CATEGORIA_ADMINISTRATIVA', u'DS_CATEGORIA_ADMINISTRATIVA', u'CO_ORGANIZACAO_ACADEMICA', u'DS_ORGANIZACAO_ACADEMICA', u'CO_LOCAL_OFERTA_CURSO', u'CO_MUNICIPIO_CURSO', u'NO_MUNICIPIO_CURSO', u'CO_UF_CURSO', u'SGL_UF_CURSO', u'NO_REGIAO_CURSO', u'IN_CAPITAL_CURSO', u'CO_CURSO', u'NO_CURSO', u'CO_SITUACAO_CURSO', u'DS_SITUACAO_CURSO', u'CO_OCDE', u'NO_OCDE', u'CO_OCDE_AREA_GERAL', u'NO_OCDE_AREA_GERAL', u'CO_OCDE_AREA_ESPECIFICA', u'NO_OCDE_AREA_ESPECIFICA', u'CO_OCDE_AREA_DETALHADA', u'NO_OCDE_AREA_DETALHADA', u'CO_GRAU_ACADEMICO', u'DS_GRAU_ACADEMICO', u'CO_MODALIDADE_ENSINO', u'DS_MODALIDADE_ENSINO', u'CO_NIVEL_ACADEMICO', u'DS_NIVEL_ACADEMICO', u'IN_GRATUITO', u'TP_ATRIBUTO_INGRESSO', u'NU_CARGA_HORARIA', u'DT_INICIO_FUNCIONAMENTO', u'DT_AUTORIZACAO_CURSO', u'IN_AJUDA_DEFICIENTE', u'IN_MATERIAL_DIGITAL', u'IN_MATERIAL_AMPLIADO', u'IN_MATERIAL_TATIL', u'IN_MATERIAL_IMPRESSO', u'IN_MATERIAL_AUDIO', u'IN_MATERIAL_BRAILLE', u'IN_MATERIAL_LIBRAS', u'IN_DISCIPLINA

In [48]:
Curso = namedtuple('Curso',headeritems) # gera a namedtuple Curso com os campos de header
print Curso._fields

('CO_IES', 'NO_IES', 'CO_CATEGORIA_ADMINISTRATIVA', 'DS_CATEGORIA_ADMINISTRATIVA', 'CO_ORGANIZACAO_ACADEMICA', 'DS_ORGANIZACAO_ACADEMICA', 'CO_LOCAL_OFERTA_CURSO', 'CO_MUNICIPIO_CURSO', 'NO_MUNICIPIO_CURSO', 'CO_UF_CURSO', 'SGL_UF_CURSO', 'NO_REGIAO_CURSO', 'IN_CAPITAL_CURSO', 'CO_CURSO', 'NO_CURSO', 'CO_SITUACAO_CURSO', 'DS_SITUACAO_CURSO', 'CO_OCDE', 'NO_OCDE', 'CO_OCDE_AREA_GERAL', 'NO_OCDE_AREA_GERAL', 'CO_OCDE_AREA_ESPECIFICA', 'NO_OCDE_AREA_ESPECIFICA', 'CO_OCDE_AREA_DETALHADA', 'NO_OCDE_AREA_DETALHADA', 'CO_GRAU_ACADEMICO', 'DS_GRAU_ACADEMICO', 'CO_MODALIDADE_ENSINO', 'DS_MODALIDADE_ENSINO', 'CO_NIVEL_ACADEMICO', 'DS_NIVEL_ACADEMICO', 'IN_GRATUITO', 'TP_ATRIBUTO_INGRESSO', 'NU_CARGA_HORARIA', 'DT_INICIO_FUNCIONAMENTO', 'DT_AUTORIZACAO_CURSO', 'IN_AJUDA_DEFICIENTE', 'IN_MATERIAL_DIGITAL', 'IN_MATERIAL_AMPLIADO', 'IN_MATERIAL_TATIL', 'IN_MATERIAL_IMPRESSO', 'IN_MATERIAL_AUDIO', 'IN_MATERIAL_BRAILLE', 'IN_MATERIAL_LIBRAS', 'IN_DISCIPLINA_LIBRAS', 'IN_TRADUTOR_LIBRAS', 'IN_GUIA_INTE

In [51]:
REGEX = r',(?=(?:[^"]*"[^"]*")*(?![^"]*"))'
# buscar por "," tal que após essa vírgula (?=) ou exista um par de "" ou não tenha " sozinha
# ?= indica para procurarmos pelo padrão após a vírgula
# ?: significa para não interpretar os parênteses como captura de valores
# [^"]* 0 ou sequências de caracteres que não sejam aspas
# [^"]*"[^"]*"  <qualquer caracter exceto aspas> " <qualquer caracter exceto aspas> "
# ?! indica para verificar se não existe tal padrão a frente da vírgula


def ParseCurso(rec):
    # utilizando re.split() vamos capturar nossos valores
    CO_IES, NO_IES, CO_CATEGORIA_ADMINISTRATIVA, DS_CATEGORIA_ADMINISTRATIVA, CO_ORGANIZACAO_ACADEMICA, DS_ORGANIZACAO_ACADEMICA, CO_LOCAL_OFERTA_CURSO, CO_MUNICIPIO_CURSO, NO_MUNICIPIO_CURSO, CO_UF_CURSO, SGL_UF_CURSO, NO_REGIAO_CURSO, IN_CAPITAL_CURSO, CO_CURSO, NO_CURSO, CO_SITUACAO_CURSO, DS_SITUACAO_CURSO, CO_OCDE, NO_OCDE, CO_OCDE_AREA_GERAL, NO_OCDE_AREA_GERAL, CO_OCDE_AREA_ESPECIFICA, NO_OCDE_AREA_ESPECIFICA, CO_OCDE_AREA_DETALHADA, NO_OCDE_AREA_DETALHADA, CO_GRAU_ACADEMICO, DS_GRAU_ACADEMICO, CO_MODALIDADE_ENSINO, DS_MODALIDADE_ENSINO, CO_NIVEL_ACADEMICO, DS_NIVEL_ACADEMICO, IN_GRATUITO, TP_ATRIBUTO_INGRESSO, NU_CARGA_HORARIA, DT_INICIO_FUNCIONAMENTO, DT_AUTORIZACAO_CURSO, IN_AJUDA_DEFICIENTE, IN_MATERIAL_DIGITAL, IN_MATERIAL_AMPLIADO, IN_MATERIAL_TATIL, IN_MATERIAL_IMPRESSO, IN_MATERIAL_AUDIO, IN_MATERIAL_BRAILLE, IN_MATERIAL_LIBRAS, IN_DISCIPLINA_LIBRAS, IN_TRADUTOR_LIBRAS, IN_GUIA_INTERPRETE, IN_RECURSOS_COMUNICACAO, IN_RECURSOS_INFORMATICA, IN_INTEGRAL_CURSO, IN_MATUTINO_CURSO, IN_VESPERTINO_CURSO, IN_NOTURNO_CURSO, NU_INTEGRALIZACAO_INTEGRAL, NU_INTEGRALIZACAO_MATUTINO, NU_INTEGRALIZACAO_VESPERTINO, NU_INTEGRALIZACAO_NOTURNO, NU_INTEGRALIZACAO_EAD, IN_OFERECE_DISC_SEMI_PRES, NU_PERC_CAR_HOR_SEMI_PRES, IN_POSSUI_LABORATORIO, QT_INSC_VAGAS_NOVAS_INT, QT_INSC_VAGAS_NOVAS_MAT, QT_INSC_VAGAS_NOVAS_VESP, QT_INSC_VAGAS_NOVAS_NOT, QT_INSC_VAGAS_NOVAS_EAD, QT_INSC_VAGAS_REMAN_INT, QT_INSC_VAGAS_REMAN_MAT, QT_INSC_VAGAS_REMAN_VESP, QT_INSC_VAGAS_REMAN_NOT, QT_INSC_VAGAS_REMAN_EAD, QT_INSC_VAGAS_PROG_ESP_INT, QT_INSC_VAGAS_PROG_ESP_MAT, QT_INSC_VAGAS_PROG_ESP_VESP, QT_INSC_VAGAS_PROG_ESP_NOT, QT_INSC_VAGAS_PROG_ESP_EAD, QT_VAGAS_NOVAS_INTEGRAL, QT_VAGAS_NOVAS_MATUTINO, QT_VAGAS_NOVAS_VESPERTINO, QT_VAGAS_NOVAS_NOTURNO, QT_VAGAS_NOVAS_EAD, QT_VAGAS_REMANESC_INTEGRAL, QT_VAGAS_REMANESC_MATUTINO, QT_VAGAS_REMANESC_VESPERTINO, QT_VAGAS_REMANESC_NOTURNO, QT_VAGAS_REMANESC_EAD, QT_VAGAS_PROG_ESP_INTEGRAL, QT_VAGAS_PROG_ESP_MATUTINO, QT_VAGAS_PROG_ESP_VESPERTINO, QT_VAGAS_PROG_ESP_NOTURNO, QT_VAGAS_PROG_ESP_EAD, QT_MATRICULA_CURSO, QT_CONCLUINTE_CURSO, QT_INGRESSO_CURSO, QT_INGRESSO_VAGAS_NOVAS, QT_VAGAS_TOTAIS = tuple(rec.split('|'))
    

    return Curso(CO_IES, NO_IES, CO_CATEGORIA_ADMINISTRATIVA, DS_CATEGORIA_ADMINISTRATIVA, CO_ORGANIZACAO_ACADEMICA, DS_ORGANIZACAO_ACADEMICA, CO_LOCAL_OFERTA_CURSO, CO_MUNICIPIO_CURSO, NO_MUNICIPIO_CURSO, CO_UF_CURSO, SGL_UF_CURSO, NO_REGIAO_CURSO, IN_CAPITAL_CURSO, CO_CURSO, NO_CURSO, CO_SITUACAO_CURSO, DS_SITUACAO_CURSO, CO_OCDE, NO_OCDE, CO_OCDE_AREA_GERAL, NO_OCDE_AREA_GERAL, CO_OCDE_AREA_ESPECIFICA, NO_OCDE_AREA_ESPECIFICA, CO_OCDE_AREA_DETALHADA, NO_OCDE_AREA_DETALHADA, CO_GRAU_ACADEMICO, DS_GRAU_ACADEMICO, CO_MODALIDADE_ENSINO, DS_MODALIDADE_ENSINO, CO_NIVEL_ACADEMICO, DS_NIVEL_ACADEMICO, IN_GRATUITO, TP_ATRIBUTO_INGRESSO, NU_CARGA_HORARIA, DT_INICIO_FUNCIONAMENTO, DT_AUTORIZACAO_CURSO, IN_AJUDA_DEFICIENTE, IN_MATERIAL_DIGITAL, IN_MATERIAL_AMPLIADO, IN_MATERIAL_TATIL, IN_MATERIAL_IMPRESSO, IN_MATERIAL_AUDIO, IN_MATERIAL_BRAILLE, IN_MATERIAL_LIBRAS, IN_DISCIPLINA_LIBRAS, IN_TRADUTOR_LIBRAS, IN_GUIA_INTERPRETE, IN_RECURSOS_COMUNICACAO, IN_RECURSOS_INFORMATICA, IN_INTEGRAL_CURSO, IN_MATUTINO_CURSO, IN_VESPERTINO_CURSO, IN_NOTURNO_CURSO, NU_INTEGRALIZACAO_INTEGRAL, NU_INTEGRALIZACAO_MATUTINO, NU_INTEGRALIZACAO_VESPERTINO, NU_INTEGRALIZACAO_NOTURNO, NU_INTEGRALIZACAO_EAD, IN_OFERECE_DISC_SEMI_PRES, NU_PERC_CAR_HOR_SEMI_PRES, IN_POSSUI_LABORATORIO, QT_INSC_VAGAS_NOVAS_INT, QT_INSC_VAGAS_NOVAS_MAT, QT_INSC_VAGAS_NOVAS_VESP, QT_INSC_VAGAS_NOVAS_NOT, QT_INSC_VAGAS_NOVAS_EAD, QT_INSC_VAGAS_REMAN_INT, QT_INSC_VAGAS_REMAN_MAT, QT_INSC_VAGAS_REMAN_VESP, QT_INSC_VAGAS_REMAN_NOT, QT_INSC_VAGAS_REMAN_EAD, QT_INSC_VAGAS_PROG_ESP_INT, QT_INSC_VAGAS_PROG_ESP_MAT, QT_INSC_VAGAS_PROG_ESP_VESP, QT_INSC_VAGAS_PROG_ESP_NOT, QT_INSC_VAGAS_PROG_ESP_EAD, QT_VAGAS_NOVAS_INTEGRAL, QT_VAGAS_NOVAS_MATUTINO, QT_VAGAS_NOVAS_VESPERTINO, QT_VAGAS_NOVAS_NOTURNO, QT_VAGAS_NOVAS_EAD, QT_VAGAS_REMANESC_INTEGRAL, QT_VAGAS_REMANESC_MATUTINO, QT_VAGAS_REMANESC_VESPERTINO, QT_VAGAS_REMANESC_NOTURNO, QT_VAGAS_REMANESC_EAD, QT_VAGAS_PROG_ESP_INTEGRAL, QT_VAGAS_PROG_ESP_MATUTINO, QT_VAGAS_PROG_ESP_VESPERTINO, QT_VAGAS_PROG_ESP_NOTURNO, QT_VAGAS_PROG_ESP_EAD, QT_MATRICULA_CURSO, QT_CONCLUINTE_CURSO, QT_INGRESSO_CURSO, QT_INGRESSO_VAGAS_NOVAS, QT_VAGAS_TOTAIS)


In [52]:

# Aplique a função ParseCurso para cada objeto da base
CursoHeadlessRDD = (CursoRawData
                    .filter(lambda x: x != header)
                    .map(ParseCurso)
                    )


firstClean = CursoHeadlessRDD.take(1)[0]

In [54]:
totalRecs = CursoHeadlessRDD.count()
print firstClean

Curso(CO_IES=u'1', NO_IES=u'UNIVERSIDADE FEDERAL DE MATO GROSSO', CO_CATEGORIA_ADMINISTRATIVA=u'1', DS_CATEGORIA_ADMINISTRATIVA=u'P\ufffdblica Federal', CO_ORGANIZACAO_ACADEMICA=u'1', DS_ORGANIZACAO_ACADEMICA=u'Universidade', CO_LOCAL_OFERTA_CURSO=u'105340', CO_MUNICIPIO_CURSO=u'5103403', NO_MUNICIPIO_CURSO=u'Cuiab\ufffd', CO_UF_CURSO=u'51', SGL_UF_CURSO=u'MT', NO_REGIAO_CURSO=u'Centro-Oeste', IN_CAPITAL_CURSO=u'1', CO_CURSO=u'1', NO_CURSO=u'DIREITO', CO_SITUACAO_CURSO=u'1', DS_SITUACAO_CURSO=u'Em atividade', CO_OCDE=u'380D01', NO_OCDE=u'Direito', CO_OCDE_AREA_GERAL=u'3', NO_OCDE_AREA_GERAL=u'Ci\ufffdncias sociais, neg\ufffdcios e direito', CO_OCDE_AREA_ESPECIFICA=u'38', NO_OCDE_AREA_ESPECIFICA=u'Direito', CO_OCDE_AREA_DETALHADA=u'380', NO_OCDE_AREA_DETALHADA=u'Direito', CO_GRAU_ACADEMICO=u'1', DS_GRAU_ACADEMICO=u'Bacharelado', CO_MODALIDADE_ENSINO=u'1', DS_MODALIDADE_ENSINO=u'Presencial', CO_NIVEL_ACADEMICO=u'1', DS_NIVEL_ACADEMICO=u'Gradua\ufffd\ufffdo', IN_GRATUITO=u'1', TP_ATRIBUTO

In [56]:
# Regioes com mais cursos

RegioesCountRDD = (CursoHeadlessRDD
               .map(lambda rec: (rec.NO_REGIAO_CURSO,1))
               .reduceByKey(lambda x,y: x+y)
               )
regioesCount = sorted(RegioesCountRDD.collect(), key=lambda x: -x[1])

AttributeError: 'list' object has no attribute 'take'

In [57]:
print regioesCount

[(u'Sudeste', 14864), (u'Nordeste', 6443), (u'Sul', 6267), (u'Centro-Oeste', 2979), (u'Norte', 2478), (u'', 1664)]


In [58]:
# Estados com mais cursos

EstadosCountRDD = (CursoHeadlessRDD
               .map(lambda rec: (rec.SGL_UF_CURSO,1))
               .reduceByKey(lambda x,y: x+y)
               )
EstadosCount = sorted(EstadosCountRDD.collect(), key=lambda x: -x[1])

In [60]:
print EstadosCount

[(u'SP', 8340), (u'MG', 3365), (u'RJ', 2502), (u'PR', 2409), (u'RS', 2397), (u'BA', 1696), (u'', 1664), (u'SC', 1461), (u'GO', 1095), (u'PE', 977), (u'CE', 902), (u'PA', 773), (u'AM', 744), (u'DF', 734), (u'MA', 692), (u'ES', 657), (u'MT', 635), (u'PB', 534), (u'MS', 515), (u'PI', 493), (u'RN', 490), (u'AL', 366), (u'RO', 318), (u'SE', 293), (u'TO', 253), (u'AP', 154), (u'AC', 124), (u'RR', 112)]


In [65]:
# Municipios com mais cursos

MunicipiosCountRDD = (CursoHeadlessRDD
               .map(lambda rec: (rec.NO_MUNICIPIO_CURSO,1))
               .reduceByKey(lambda x,y: x+y)
               )
MunicipiosCount = sorted(MunicipiosCountRDD.collect(), key=lambda x: -x[1])
print '\nOs 20 municipios com mais oferta de curso:\n{0}'.format(MunicipiosCount[:20])


Os20 municipios com mais oferta de curso:
[(u'S\ufffdo Paulo', 2692), (u'', 1664), (u'Rio de Janeiro', 1380), (u'Belo Horizonte', 755), (u'Curitiba', 745), (u'Bras\ufffdlia', 734), (u'Salvador', 669), (u'Fortaleza', 546), (u'Recife', 465), (u'Manaus', 458), (u'Porto Alegre', 436), (u'Goi\ufffdnia', 400), (u'Campinas', 379), (u'Bel\ufffdm', 350), (u'Teresina', 279), (u'Natal', 278), (u'S\ufffdo Lu\ufffds', 270), (u'Jo\ufffdo Pessoa', 257), (u'Macei\ufffd', 241), (u'Sorocaba', 230)]


In [66]:
# IES com mais cursos

IESCountRDD = (CursoHeadlessRDD
               .map(lambda rec: (rec.NO_IES,1))
               .reduceByKey(lambda x,y: x+y)
               )
IESCount = sorted(IESCountRDD.collect(), key=lambda x: -x[1])
print '\nAs 20 IES com mais oferta de curso:\n{0}'.format(IESCount[:20])


As 20 IES com mais oferta de curso:
[(u'UNIVERSIDADE PAULISTA', 950), (u'UNIVERSIDADE EST\ufffdCIO DE S\ufffd', 486), (u'UNIVERSIDADE DO ESTADO DO AMAZONAS', 321), (u'UNIVERSIDADE ANHANGUERA DE S\ufffdO PAULO - UNIAN-SP', 293), (u'UNIVERSIDADE NOVE DE JULHO', 272), (u'UNIVERSIDADE DE S\ufffdO PAULO', 267), (u'UNIVERSIDADE ESTADUAL DO MARANH\ufffdO', 261), (u'UNIVERSIDADE DO ESTADO DA BAHIA', 236), (u'UNIVERSIDADE ESTADUAL PAULISTA J\ufffdLIO DE MESQUITA FILHO', 219), (u'UNIVERSIDADE FEDERAL DO RIO DE JANEIRO', 198), (u'UNIVERSIDADE FEDERAL DO PARAN\ufffd', 180), (u'UNIVERSIDADE LUTERANA DO BRASIL', 174), (u'UNIVERSIDADE FEDERAL DE GOI\ufffdS', 173), (u'UNIVERSIDADE DE CAXIAS DO SUL', 162), (u'UNIVERSIDADE DE BRAS\ufffdLIA', 154), (u'UNIVERSIDADE FEDERAL DO PAR\ufffd', 152), (u'UNIVERSIDADE ESTADUAL DE GOI\ufffdS', 145), (u'CENTRO UNIVERSIT\ufffdRIO ANHANGUERA DE S\ufffdO PAULO', 143), (u'UNIVERSIDADE ESTADUAL DO PIAU\ufffd', 141), (u'UNIVERSIDADE FEDERAL DA BAHIA', 137)]
